# Input file writing

## HTCFile

### Quickstart

The HTCFile class allows you to open, modify, save htc files and run HAWC2 simulations

In [1]:
from wetb import hawc2
from wetb.hawc2 import HTCFile
from wetb.hawc2.tests.test_files import tfp
htc = HTCFile(tfp + "htcfiles/DTU_10MW_RWT.htc")
htc.wind.wsp=11
print (htc)
htc.save("tmp/tmp.htc")
# htc.simulate('<path_to_hawc2>/hawc2mb.exe')

;DTU_10MW_RWT, version 9, 25-09-2017, mhha
;
  begin simulation;
    time_stop	1000;
    solvertype	1;	(newmark)
    on_no_convergence	continue;
    convergence_limits	1000 1 1e-07;
    logfile	./log/DTU_10MW_RWT_ver09.log;
    begin newmark;
      deltat	0.01;
    end newmark;
  end simulation;
;
;----------------------------------------------------------------------------------------------------------------------------------------------------------------
  begin new_htc_structure;
;   beam_output_file_name  ./log/DTU_10MW_RWT_beam.dat;                    Optional - Calculated beam properties of the bodies are written to file
;   body_output_file_name  ./log/DTU_10MW_RWT_body.dat;                    Optional - Body initial position and orientation are written to file
;   body_eigenanalysis_file_name ./eig/DTU_10MW_RWT_body_eigen.dat;
;   structure_eigenanalysis_file_name ./eig/DTU_10MW_RWT_strc_eigen.dat ;
;------------------------------------------------------------------------------

### Load htc files
`HTCFile` takes the htc-filename and optionally the modelpath as input. 

The `modelpath` is the path from where HAWC2 should be run, i.e. the the origion for relative data file paths in the htc file. If not specified (default) the `modelpath` is autodetected if possible.

In [2]:
htc = HTCFile(filename=tfp+"htcfiles/DTU_10MW_RWT.htc",
              modelpath=None # modelpath autodetected
             )
print (htc.modelpath)

C:/mmpe/programming/python/WindEnergyToolbox/wetb/hawc2/tests/test_files/


### Access htc contents

Access elements(lines and sections) in the htc file via dot or string notation

In [3]:
wsp = htc.wind.wsp # dot notation
print (wsp)
convergence_limits = htc['simulation.convergence_limits'] # string notation with dot
print (convergence_limits)

wsp	4;

convergence_limits	1000 1 1e-07;



you can also store a (sub)section in a varble and access subelements from that

In [4]:
wind = htc.wind
print (wind.wsp)

wsp	4;



Values of a line are obtained by

In [5]:
print (wsp[0])
print (htc.simulation.convergence_limits[:])

4
[1000, 1, 1e-07]


In case there is more than one element with the same name, e.g. main_body sections in new_htc_structure:

In [6]:
htc.new_htc_structure.keys()

['main_body',
 'main_body__2',
 'main_body__3',
 'main_body__4',
 'main_body__5',
 'main_body__6',
 'main_body__7',
 'main_body__8',
 'main_body__9',
 'orientation',
 'constraint']

Then the right element can be access by adding `__x` (where x is the section number).
The first element can be accessed without `__x` or with `__1` 


In [7]:
htc.new_htc_structure.main_body__1 == htc.new_htc_structure.main_body

True

Another way to access these elements are:

In [8]:
htc.new_htc_structure.get_subsection_by_name('shaft')
htc.new_htc_structure.main_body(name='shaft')

#### HTC Section

In [9]:
section = htc.new_htc_structure.main_body__3.c2_def
print(section.section_name)
print(section.keys())
print(section.parent.section_name)
print(section.parent.location())
print()
print(section)


c2_def
['nsec', 'sec', 'sec__2', 'sec__3', 'sec__4', 'sec__5']
main_body
DTU_10MW_RWT.htc/new_htc_structure/main_body__3

begin c2_def;	Definition of centerline (main_body coordinates)
  nsec	5;
  sec	1 0 0 0 0;	Tower top x,y,z,twist
  sec	2 0 0 1.5 0;
  sec	3 0 0 3 0;
  sec	4 0 0 4.4 0;	Main bearing
  sec	5 0 0 7.1 0;	Rotor centre
end c2_def;



#### HTC Line

In [10]:
line = htc.new_htc_structure.main_body__3.c2_def.sec__4

print(line.values) # print all values in line
print(line.values[3]) # print value 4 (zero-indexed)
print(line.comments) # print comments
print()
print(line) # print line

[4, 0, 0, 4.4, 0]
4.4
Main bearing

sec	4 0 0 4.4 0;	Main bearing



#### Output sensor

In [11]:
print (htc.output.sensors[3])
print (htc.output.sensors[3].type)
print (htc.output.sensors[3].sensor)
print (htc.output.sensors[3].values)
print (htc.output.sensors[3].comments)


constraint bearing2	pitch2 5;	angle and angle velocity

constraint
bearing2
['pitch2', 5]
angle and angle velocity


### Add htc contents
#### Add htc line

In [12]:
htc.simulation.my_line = "value1", 2
htc.simulation.my_line.comments = "MyComment"
print (htc.simulation)

begin simulation;
  time_stop	1000;
  solvertype	1;	(newmark)
  on_no_convergence	continue;
  convergence_limits	1000 1 1e-07;
  logfile	./log/DTU_10MW_RWT_ver09.log;
  begin newmark;
    deltat	0.01;
  end newmark;
  my_line	value1 2;	MyComment
end simulation;
;
;----------------------------------------------------------------------------------------------------------------------------------------------------------------



In case there is already one line with the same key, then you need to use the `add_line` function

In [13]:
htc.simulation.add_line(name="my_line", 
                        values=["value1", 2], 
                        comments="MyComment for second my_line" #defaults to ""
                       )
print (htc.simulation)

begin simulation;
  time_stop	1000;
  solvertype	1;	(newmark)
  on_no_convergence	continue;
  convergence_limits	1000 1 1e-07;
  logfile	./log/DTU_10MW_RWT_ver09.log;
  begin newmark;
    deltat	0.01;
  end newmark;
  my_line	value1 2;	MyComment
  my_line	value1 2;	MyComment for second my_line
end simulation;
;
;----------------------------------------------------------------------------------------------------------------------------------------------------------------



#### Add htc section

In [14]:
my_section = htc.simulation.add_section(section_name="my_section", 
                           members={"Line1":1, "line2":[21,22,23]}, 
                           allow_duplicate=False,
                           line3=[31,32])
print (htc.simulation)

begin simulation;
  time_stop	1000;
  solvertype	1;	(newmark)
  on_no_convergence	continue;
  convergence_limits	1000 1 1e-07;
  logfile	./log/DTU_10MW_RWT_ver09.log;
  begin newmark;
    deltat	0.01;
  end newmark;
  my_line	value1 2;	MyComment
  my_line	value1 2;	MyComment for second my_line
  begin my_section;
    line3	31 32;
    Line1	1;
    line2	21 22 23;
  end my_section;
end simulation;
;
;----------------------------------------------------------------------------------------------------------------------------------------------------------------



If `allow_duplicate` is `False` and a section with the same name already exists, then `add_section` has no effect

In [15]:
my_section = htc.simulation.add_section(section_name="my_section", 
                                        allow_duplicate=False,
                                        line4=4)
print (htc.simulation)

begin simulation;
  time_stop	1000;
  solvertype	1;	(newmark)
  on_no_convergence	continue;
  convergence_limits	1000 1 1e-07;
  logfile	./log/DTU_10MW_RWT_ver09.log;
  begin newmark;
    deltat	0.01;
  end newmark;
  my_line	value1 2;	MyComment
  my_line	value1 2;	MyComment for second my_line
  begin my_section;
    line3	31 32;
    Line1	1;
    line2	21 22 23;
  end my_section;
end simulation;
;
;----------------------------------------------------------------------------------------------------------------------------------------------------------------



#### Add output sensor

In [16]:
htc.output.add_sensor('sensor_type', 'sensor_name', ['arg1', 2], comment = "MySensor", nr=3)
print (htc.output)

begin output;
  filename	./res/DTU_10MW_RWT_ver09;
  data_format	hawc_binary;
  buffer	1;	
;
  general time;
  constraint bearing1	shaft_rot 2;	angle and angle velocity
  constraint bearing2	pitch1 5;	angle and angle velocity
  sensor_type sensor_name	arg1 2;	MySensor
  constraint bearing2	pitch2 5;	angle and angle velocity
  constraint bearing2	pitch3 5;	angle and angle velocity
  aero omega;
  aero torque;
  aero power;
  aero thrust;
  wind free_wind	1 0 0 -119;	local wind at fixed position: coo (1=global,2=non-rotation rotor coo.), pos x, pos y, pos z
; Moments:
  mbdy momentvec	tower 1 1 tower # tower base;
  mbdy momentvec	tower 10 2 tower # tower yaw bearing;
  mbdy momentvec	shaft 4 1 shaft # main bearing;
  mbdy momentvec	blade1 2 2 blade1 # blade 1 root;
  mbdy momentvec	blade2 2 2 blade2 # blade 2 root;
  mbdy momentvec	blade3 2 2 blade3 # blade 3 root;
  mbdy momentvec	blade1 13 1 local # blade 1 50% local e coo;
  mbdy momentvec	blade2 13 1 local # blade 2 50% local e coo;

### Modify htc contents

In [18]:
htc.wind.wsp = 12 # set single value element
print (htc.wind.wsp)

c2def = htc.new_htc_structure.main_body__3.c2_def
c2def.section_name = "c2_def_new"
c2def.sec__4 = 1,2,3,4  # set element with four values
c2def.sec__4[3] = 9
print (c2def)

wsp	12;

begin c2_def_new;	Definition of centerline (main_body coordinates)
  nsec	5;
  sec	1 0 0 0 0;	Tower top x,y,z,twist
  sec	2 0 0 1.5 0;
  sec	3 0 0 3 0;
  sec	1 2 3 9;	Main bearing
  sec	5 0 0 7.1 0;	Rotor centre
end c2_def_new;



Note, that you cannot change modify values of the current line, e.g. 
```python
line = htc.new_htc_structure.main_body__3.c2_def.sec__4
line = 1,2,3,4
```
as it will overwrite the variable `line` with the tuple (1,2,3,4) instead of updating the values

### Delete
Sections and lines are deleted by calling their `delete` method

In [44]:
htc.simulation.my_line.delete() # second my_line (my_line__2) remains
htc.simulation.my_section.delete()
print (htc.simulation)

begin simulation;
  time_stop	1000;
  solvertype	1;	(newmark)
  on_no_convergence	continue;
  convergence_limits	1000 1 1e-07;
  logfile	./log/DTU_10MW_RWT_ver09.log;
  begin newmark;
    deltat	0.01;
  end newmark;
  my_line	value1 2;	MyComment for second my_line
end simulation;
;
;----------------------------------------------------------------------------------------------------------------------------------------------------------------



### HTC functions
#### set_name()
The `set_name` function sets the filename, as well as the

- log filename
- animation filename
- visualization filename
- output filename (only first output section)

In [45]:
htc.set_name(name="new_name", subfolder='my_folder')
print (htc.filename)
print (htc.simulation.logfile)
print (htc.output.filename)

c:/mmpe/programming/python/windenergytoolbox/wetb/hawc2/tests/test_files/htc/my_folder/new_name.htc
logfile	./log/my_folder/new_name.log;

filename	./res/my_folder/new_name;



#### set_time()
The `set_time(start=None, stop=None, step=None)` function sets the start, stop and step time if specified:

In [46]:
htc.set_time(10,20,.1)
print (htc.simulation.time_stop)
print (htc.simulation.newmark.deltat)
print (htc.wind.scale_time_start)
print (htc.output.time)


time_stop	20;

deltat	0.1;

scale_time_start	10;

time	10 20;



#### simulate()
The `simulate(exe)` function saves the htc file, performs the simulation using the specified exe (HAWC2), raises an error if the simulation fails and returns the console output and log

```python
stdout, log = htc.simulate("<path_to_hawc2>\hawc2mb.exe")
```

# Parameter study

HTC files for parameter studies, can be generated by a loop that updates the desired values of the htc and saves the result:

In [49]:
from wetb.hawc2 import HTCFile
from wetb.hawc2.tests.test_files import tfp
htc = HTCFile(tfp + "htcfiles/DTU_10MW_RWT.htc")

for wsp in [4,6,8]:
    htc.wind.wsp = wsp
    htc.set_name("case_wsp%d"%wsp)
    htc.save()

## HAWC2InputWriter

An easier way for more complicate studies is to use the HAWC2InputWriter which takes a case as input (see [Case list generation](#Case-list-generation))

A case is dictionary of (key,value)-pairs. Keys can be:

1. Htc address, e.g. wind.wsp
2. Function name, e.g. `wsp` in which case `value` is passed to the `set_wsp` of `HAWC2InputWriter` (which you must implement)
3. A tag that is replaced by `value`

Note, all three types can be combined

First we create the htc file that we want the parameter study to be based on

In [50]:
%%writefile "tmp/base.htc"

begin simulation;
    time_stop	600;
    logfile log.txt
end simulation;
;
begin wind;
    wsp	10;
end wind;
exit;

Writing tmp/base.htc


### htc address

For (key-value) cases where the keys are htc addresses, we just need a `HAWC2InputWriter`

In [51]:
from wetb.hawc2 import HTCFile
from wetb.hawc2 import HAWC2InputWriter
h2writer = HAWC2InputWriter(base_htc_file='tmp/base.htc')

that allow us to make a case like this

In [53]:
h2writer('tmp/my_case.htc', **{'wind.wsp':12, 'simulation.logfile': "my_case.log"})

In [55]:
print (HTCFile('tmp/my_case.htc'))

  begin simulation;
    time_stop	600;
    logfile	my_case.log;
  end simulation;
;
  begin wind;
    wsp	12;
  end wind;
exit;


### Function name

For (key-value) cases where the keys are function names, a subclass of `HAWC2InputWriter` with the required functions must be implemented:

In [56]:
class MyInputWriter(HAWC2InputWriter):
    def set_wsp(self, htc, wsp, **_):
        htc.wind.wsp = wsp
        bs = self.begin_step
        for i, o in enumerate([0,50],1):
            htc.wind.add_line("wind_ramp_abs", [bs+o, bs+o+1, 0, 1], "wsp. after the step:  %d.0"%(wsp+i))
        # ...
        
    def set_log(self, htc, log, **_):
        if log:
            htc.simulation.logfile = log
        else:
            htc.simulation.logfile.delete()

This approach enables

- Multiple values, e.g. wind speed, turbulence grid size, wind ramp etc., can be set by one (key,value)-pair
- Lines or sections can be deleted
- Use of constants (specified when creating the writer)
- Modification of e.g. aerodynamic, profile or structural data files

In [57]:
h2writer = MyInputWriter(base_htc_file='tmp/base.htc',
                         begin_step=100, # constant need in the implemented functions (specify as many as you want)
                        )
h2writer('tmp/my_case.htc', wsp=12, log=None, sensor_y=[10,20])

In [58]:
print (HTCFile('tmp/my_case.htc'))

  begin simulation;
    time_stop	600;
  end simulation;
;
  begin wind;
    wsp	12;
    wind_ramp_abs	100 101 0 1;	wsp. after the step:  13.0
    wind_ramp_abs	150 151 0 1;	wsp. after the step:  14.0
  end wind;
exit;


### Tags

The tag options offers template rendering via the [jinja](https://en.wikipedia.org/wiki/Jinja_(template_engine)) templating language. Jinja templates allows:

- variable substitution,
```
wsp {{wsp}}
```
    
- conditionals
```
{% if logfilename %}
    logfile  {{logfilename}}
{% endif %} 
```

- loops
```
{%- for y in sensor_y %}
   wind free_wind 1 0.0 {{ y }} -119;
{%- endfor %}
```

let us make a new template.htc


In [60]:
%%writefile "tmp/template.htc"

begin simulation;
    time_stop	600;
    {% if logfilename %}
      logfile  {{logfilename}}
    {% endif %}
end simulation;
;
begin wind;
    wsp	{{wsp}};
    {%- for i in range(2) %}
        wind_ramp_abs	{{begin_step+i*50}} {{begin_step+i*50+1}} 0 1;	wsp. after the step:  {{wsp+i+1}}
    {%- endfor %}
end wind;
exit;

Writing tmp/template.htc


In [61]:
h2writer = HAWC2InputWriter(base_htc_file='tmp/template.htc')
h2writer('tmp/my_case.htc', wsp=12, log=None, begin_step=100)

In [62]:
print (HTCFile('tmp/my_case.htc'))

  begin simulation;
    time_stop	600;
  end simulation;
;
  begin wind;
    wsp	12;
    wind_ramp_abs	100 101 0 1;	wsp. after the step:  13
    wind_ramp_abs	150 151 0 1;	wsp. after the step:  14
  end wind;
exit;


## Case list generation

The `HAWC2InputWriter` can also handle a list of cases. 

These cases can be defined in a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) or an excel sheet

Note, case lists must contain a `Name` column

### Pandas Dataframe

In [63]:
import pandas as pd
df = pd.DataFrame({'Name':['case10', 'case12'], 'wsp':[10,12], 'log':['','log.txt']})
df

,Name,wsp,log
0,case10,10,
1,case12,12,log.txt


In [64]:
h2writer = MyInputWriter(base_htc_file='tmp/base.htc',
                         begin_step=100, # constant need in the implemented functions (specify as many as you want)
                        )
h2writer.from_pandas(df)
h2writer.write_all(out_dir="tmp")

Generating 2 htc files in directory: tmp


In [65]:
print(HTCFile('tmp/case10.htc'))

  begin simulation;
    time_stop	600;
  end simulation;
;
  begin wind;
    wsp	10;
    wind_ramp_abs	100 101 0 1;	wsp. after the step:  11.0
    wind_ramp_abs	150 151 0 1;	wsp. after the step:  12.0
  end wind;
exit;


In [66]:
print(HTCFile('tmp/case12.htc'))

  begin simulation;
    time_stop	600;
    logfile	log.txt;
  end simulation;
;
  begin wind;
    wsp	12;
    wind_ramp_abs	100 101 0 1;	wsp. after the step:  13.0
    wind_ramp_abs	150 151 0 1;	wsp. after the step:  14.0
  end wind;
exit;


### Excel

Saving previous dataframe as Excel spreadsheet

In [69]:
df.to_excel('tmp/caselist.xlsx', index=False)

which can be opened and modified in Excel

In [70]:
# uncomment to open in excel
# %system "tmp/caselist.xlsx"

Loading case list from (modifyed) excel sheet and write htc files

In [71]:
h2writer.from_excel('tmp/caselist.xlsx')
h2writer.contents
h2writer.write_all('tmp')

Generating 2 htc files in directory: tmp


# Design Load Basis (DLB)

The `DTU_IEC64100_1_Ref_DLB` can make a case list covering a subset of the full DLB as defined in IEC64100-1(2005). 

Note, some DLCs are missing.

Some interpretation are made see [Hansen, M. H., Thomsen, K., Natarajan, A., & Barlas, A. (2015). Design Load Basis for onshore turbines - Revision 00. DTU Wind Energy. DTU Wind Energy E, No. 0074(EN)](https://backend.orbit.dtu.dk/ws/portalfiles/portal/106567720/Design_Load_Basis_for_onshore_turbines.pdf)

In [72]:
from wetb.dlb.iec64100_1 import DTU_IEC64100_1_Ref_DLB, DLB, DLC
dlb = DTU_IEC64100_1_Ref_DLB(iec_wt_class='1A', #IEC wind turbine class
                             Vin=4, # cut-in wind speed
                             Vout=25, # cut-out wind speed
                             Vr=8, # rated wind speed
                             D=180, # Rotor diameter
                             z_hub=110 # hub height
                            )

## DLB definition
You can work with a DLB on an definition level

In [73]:
dlb.dlcs

,Name,Description,WSP,Wdir,Turb,Seeds,Shear,Gust,Fault,Time
DLC12,DLC12,Normal production,Vin:2:Vout,-10/0/10,NTM,6.0,NWP,None,None,600
DLC13,DLC13,Normal production with high turbulence,Vin:2:Vout,-10/0/10,ETM,6.0,NWP,None,None,600
DLC14,DLC14,Normal production with gust and direction change,Vr/Vr+2/Vr-2,0,NoTurb,NaN,NWP,ECD,None,100
DLC15,DLC15,Normal production with extreme wind shear,Vin:2:Vout,0,NoTurb,NaN,EWS,None,None,100
DLC21,DLC21,Loss of electical network,Vin:2:Vout,-10/0/10,NTM,4.0,NWP,None,GridLoss10,100
DLC22y,DLC22y,Abnormal yaw error,Vin:2:Vout,15:15:345,NTM,1.0,NWP,None,None,600


In [74]:
dlb.variables

,Name,Value,Description
Vin,Vin,4,Cut-in wind speed
Vout,Vout,25,Cut-out wind speed
Vr,Vr,8,Rated wind speed
D,D,180,Rotor diameter
z_hub,z_hub,110,Hub height
Vstep,Vstep,2,Wind speed distribution step
iec_wt_class,iec_wt_class,1A,"IEC wind turbine class, e.g. 1A"


### Edit DLB definition

`dlb.dlcs` is a Pandas DataFrame that you can edit in python

In [75]:
dlb.dlcs.loc['DLC12','Seeds'] = 1
dlb.dlcs


,Name,Description,WSP,Wdir,Turb,Seeds,Shear,Gust,Fault,Time
DLC12,DLC12,Normal production,Vin:2:Vout,-10/0/10,NTM,1.0,NWP,None,None,600
DLC13,DLC13,Normal production with high turbulence,Vin:2:Vout,-10/0/10,ETM,6.0,NWP,None,None,600
DLC14,DLC14,Normal production with gust and direction change,Vr/Vr+2/Vr-2,0,NoTurb,NaN,NWP,ECD,None,100
DLC15,DLC15,Normal production with extreme wind shear,Vin:2:Vout,0,NoTurb,NaN,EWS,None,None,100
DLC21,DLC21,Loss of electical network,Vin:2:Vout,-10/0/10,NTM,4.0,NWP,None,GridLoss10,100
DLC22y,DLC22y,Abnormal yaw error,Vin:2:Vout,15:15:345,NTM,1.0,NWP,None,None,600


`dlb.variables` is a Pandas DataFrame that you can edit in python

In [76]:
dlb.variables.loc['Vin','Value'] = 8
dlb.variables.loc['Vout', 'value']=10
dlb.variables

,Name,Value,Description,value
Vin,Vin,8,Cut-in wind speed,NaN
Vout,Vout,25,Cut-out wind speed,10.0
Vr,Vr,8,Rated wind speed,NaN
D,D,180,Rotor diameter,NaN
z_hub,z_hub,110,Hub height,NaN
Vstep,Vstep,2,Wind speed distribution step,NaN
iec_wt_class,iec_wt_class,1A,"IEC wind turbine class, e.g. 1A",NaN


### Edit DLB definition in Excel

You can also save the DLB definition and variables as an Excel workbook

In [78]:
dlb.to_excel('tmp/overview.xlsx')

open, edit and save in Excel

In [7]:
# uncomment to open file in excel
# %system "tmp/overview.xlsx"

and load the modified DLB

In [79]:
dlb = DLB.from_excel('tmp/overview.xlsx')
dlb.dlcs

,Name,Description,WSP,Wdir,Turb,Seeds,Shear,Gust,Fault,Time
DLC12,DLC12,Normal production,Vin:2:Vout,-10/0/10,NTM,1.0,NWP,NaN,NaN,600
DLC13,DLC13,Normal production with high turbulence,Vin:2:Vout,-10/0/10,ETM,6.0,NWP,NaN,NaN,600
DLC14,DLC14,Normal production with gust and direction change,Vr/Vr+2/Vr-2,0,NoTurb,NaN,NWP,ECD,NaN,100
DLC15,DLC15,Normal production with extreme wind shear,Vin:2:Vout,0,NoTurb,NaN,EWS,NaN,NaN,100
DLC21,DLC21,Loss of electical network,Vin:2:Vout,-10/0/10,NTM,4.0,NWP,NaN,GridLoss10,100
DLC22y,DLC22y,Abnormal yaw error,Vin:2:Vout,15:15:345,NTM,1.0,NWP,NaN,NaN,600


## DLC cases

You can generate a Pandas DataFrame with all cases of all DLCs

In [80]:
dlb.to_pandas()

,DLC,Name,Folder,V_hub,wdir,simulation_time,ti,seed,shear,Gust,Fault
0,DLC12,DLC12_wsp08_wdir350_s1001,DLC12,8.0,-10.0,600,0.232000,1001,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
1,DLC12,DLC12_wsp08_wdir000_s1001,DLC12,8.0,0.0,600,0.232000,1001,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
2,DLC12,DLC12_wsp08_wdir010_s1001,DLC12,8.0,10.0,600,0.232000,1001,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
3,DLC12,DLC12_wsp10_wdir350_s1101,DLC12,10.0,-10.0,600,0.209600,1101,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
4,DLC12,DLC12_wsp10_wdir000_s1101,DLC12,10.0,0.0,600,0.209600,1101,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
225,DLC22y,DLC22y_wsp26_wdir285_s1901,DLC22y,26.0,285.0,600,0.154462,1901,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
226,DLC22y,DLC22y_wsp26_wdir300_s1901,DLC22y,26.0,300.0,600,0.154462,1901,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
227,DLC22y,DLC22y_wsp26_wdir315_s1901,DLC22y,26.0,315.0,600,0.154462,1901,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
228,DLC22y,DLC22y_wsp26_wdir330_s1901,DLC22y,26.0,330.0,600,0.154462,1901,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN


or the cases of a single DLC

In [81]:
dlb['DLC14']

,DLC,Name,Folder,V_hub,wdir,simulation_time,seed,shear,Gust
0,DLC14,DLC14_wsp06_wdir000,DLC14,6.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 120.0,..."
1,DLC14,DLC14_wsp08_wdir000,DLC14,8.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 90.0, ..."
2,DLC14,DLC14_wsp10_wdir000,DLC14,10.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 72.0, ..."


### Edit DLC cases

The dlc cases returned by `dlb.dlcs[<dlc>]` is a Pandas DataFrame that you can edit in python

!!! Note, editing dlc cases will not influence the DLB

In [82]:
dlc14_ed = dlb['DLC14']
dlc14_ed.loc[0,'V_hub'] = 4
dlc14_ed

,DLC,Name,Folder,V_hub,wdir,simulation_time,seed,shear,Gust
0,DLC14,DLC14_wsp06_wdir000,DLC14,4.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 120.0,..."
1,DLC14,DLC14_wsp08_wdir000,DLC14,8.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 90.0, ..."
2,DLC14,DLC14_wsp10_wdir000,DLC14,10.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 72.0, ..."


### Edit DLC cases in Excel

You can also save the DLC case list as an Excel spreadsheet

In [83]:
dlc14_ed.to_excel('tmp/dlc14_ed.xlsx')

and open and edit the cases in Excel

In [84]:
# uncomment to open file in excel
# %system "tmp/dlc14_ed.xlsx"

The modified cases can be loaded by the input writer, see below

### Write DLB/DLC cases as HAWC2 input files

#### Load into HAWC2_IEC_DLC_Writer

To write the HTC files correponding to the DLB cases, you need the `HAWC2_IEC_DLC_Writer` and a base htc file

In [85]:
from wetb.hawc2.tests.test_files import tfp
from wetb.dlb.hawc2_iec_dlc_writer import HAWC2_IEC_DLC_Writer

path = tfp + '/simulation_setup/DTU10MWRef6.0/'
writer = HAWC2_IEC_DLC_Writer(path + 'htc/DTU_10MW_RWT.htc', diameter=180)

Load all cases into the writer

In [86]:
writer.from_pandas(dlb)
writer.contents

,DLC,Name,Folder,V_hub,wdir,simulation_time,ti,seed,shear,Gust,Fault
0,DLC12,DLC12_wsp08_wdir350_s1001,DLC12,8.0,-10.0,600,0.232000,1001,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
1,DLC12,DLC12_wsp08_wdir000_s1001,DLC12,8.0,0.0,600,0.232000,1001,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
2,DLC12,DLC12_wsp08_wdir010_s1001,DLC12,8.0,10.0,600,0.232000,1001,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
3,DLC12,DLC12_wsp10_wdir350_s1101,DLC12,10.0,-10.0,600,0.209600,1101,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
4,DLC12,DLC12_wsp10_wdir000_s1101,DLC12,10.0,0.0,600,0.209600,1101,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
225,DLC22y,DLC22y_wsp26_wdir285_s1901,DLC22y,26.0,285.0,600,0.154462,1901,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
226,DLC22y,DLC22y_wsp26_wdir300_s1901,DLC22y,26.0,300.0,600,0.154462,1901,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
227,DLC22y,DLC22y_wsp26_wdir315_s1901,DLC22y,26.0,315.0,600,0.154462,1901,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN
228,DLC22y,DLC22y_wsp26_wdir330_s1901,DLC22y,26.0,330.0,600,0.154462,1901,"{'type': 'NWP', 'profile': ('power', 0.2)}",NaN,NaN


Load a single DLC

In [87]:
writer.from_pandas(dlb['DLC14'])
writer.contents

,DLC,Name,Folder,V_hub,wdir,simulation_time,seed,shear,Gust
0,DLC14,DLC14_wsp06_wdir000,DLC14,6.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 120.0,..."
1,DLC14,DLC14_wsp08_wdir000,DLC14,8.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 90.0, ..."
2,DLC14,DLC14_wsp10_wdir000,DLC14,10.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 72.0, ..."


Load the modified DLC

In [88]:
writer.from_pandas(dlc14_ed)
writer.contents

,DLC,Name,Folder,V_hub,wdir,simulation_time,seed,shear,Gust
0,DLC14,DLC14_wsp06_wdir000,DLC14,4.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 120.0,..."
1,DLC14,DLC14_wsp08_wdir000,DLC14,8.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 90.0, ..."
2,DLC14,DLC14_wsp10_wdir000,DLC14,10.0,0.0,100,None,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 72.0, ..."


Load the DLC from the (modified) excel file

In [89]:
writer.from_excel('tmp/dlc14_ed.xlsx')
writer.contents

,DLC,Name,Folder,V_hub,wdir,simulation_time,seed,shear,Gust
0,DLC14,DLC14_wsp06_wdir000,DLC14,4,0,100,,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 120.0,..."
1,DLC14,DLC14_wsp08_wdir000,DLC14,8,0,100,,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 90.0, ..."
2,DLC14,DLC14_wsp10_wdir000,DLC14,10,0,100,,"{'type': 'NWP', 'profile': ('power', 0.2)}","{'type': 'ECD', 'V_cg': 15, 'theta_cg': 72.0, ..."


#### Write files

In [90]:
writer.write_all(out_dir='tmp')

Generating 3 htc files in directory: tmp


In [91]:
%ls "tmp/DLC14"

 Volume in drive C is Windows
 Volume Serial Number is 1AB2-743B

 Directory of c:\mmpe\programming\python\WindEnergyToolbox\notebooks\hawc2\tmp\DLC14

29-04-2020  14:56    <DIR>          .
29-04-2020  14:56    <DIR>          ..
29-04-2020  14:56            28,439 DLC14_wsp06_wdir000.htc
29-04-2020  14:56            28,438 DLC14_wsp08_wdir000.htc
29-04-2020  14:56            28,439 DLC14_wsp10_wdir000.htc
               3 File(s)         85,316 bytes
               2 Dir(s)  56,159,342,592 bytes free
